In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from setuptools import glob
# Colab cell 2: install deps
# 3) Install / upgrade 🤗 transformers + datasets
!pip install -q transformers datasets
from transformers.trainer_utils import get_last_checkpoint


In [ ]:
!ls ./ToxicCommentClassifier_Final/artifacts/

models	test.csv  train.csv


In [ ]:
! ls ./ToxicCommentClassifier_Final/


artifacts	 notebook	      raw_file		streamlit_app.py
Dockerfile	 predict.py	      requirements.txt	streamlit_report.py
export_model.py  projectstructure.py  setup.py		train.py
main.py		 pyproject.toml       src


In [ ]:
# from transformers import BertForSequenceClassification, BertTokenizerFast
# import os
# 
# DRIVE_DIR = "./ToxicCommentClassifier_Final/artifacts/models/bert_toxic"
# os.makedirs(DRIVE_DIR, exist_ok=True)
# 
# # 1) load a known-good checkpoint (here we use the HF base model)
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
# 
# # 2) immediately save it into your artifacts folder:
# model.save_pretrained(DRIVE_DIR)
# tokenizer.save_pretrained(DRIVE_DIR)
# 
# print("✓ Re-exported model & tokenizer to", DRIVE_DIR)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

✓ Re-exported model & tokenizer to /content/drive/MyDrive/ToxicCommentClassifier_Final/artifacts/models/bert_toxic


In [ ]:
# !ls ./ToxicCommentClassifier_Final/artifacts/models/bert_toxic


checkpoint-150	config.json	   runs			    tokenizer.json
checkpoint-200	model.safetensors  special_tokens_map.json  vocab.txt
checkpoint-250	pytorch_model.bin  tokenizer_config.json


In [ ]:
# Colab cell 4: path setup
# import os
# BASE = "./ToxicCommentClassifier_Final"
# ARTIFACTS = os.path.join(BASE, "artifacts")
# DATA_CSV = os.path.join(ARTIFACTS, "train.csv")
# MODEL_DIR = os.path.join(ARTIFACTS, "models", "bert_toxic")
# 
# print("→ data:", DATA_CSV)
# print("→ model dir:", MODEL_DIR)
# assert os.path.exists(DATA_CSV), "train.csv not found!"
# os.makedirs(MODEL_DIR, exist_ok=True)


→ data: /content/drive/MyDrive/ToxicCommentClassifier_Final/artifacts/train.csv
→ model dir: /content/drive/MyDrive/ToxicCommentClassifier_Final/artifacts/models/bert_toxic


In [ ]:
# from transformers.trainer_utils import get_last_checkpoint
# from transformers import BertForSequenceClassification
# 
# def load_model(path):
#     try:
#         print("→ loading from", path)
#         return BertForSequenceClassification.from_pretrained(path, local_files_only=True, num_labels=6)
#     except Exception as e:
#         print("⚠️ failed:", e)
#         print("→ falling back to bert-base-uncased")
#         return BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)
# 
# model = load_model("./ToxicCommentClassifier_Final/artifacts/models/bert_toxic")


→ loading from /content/drive/MyDrive/ToxicCommentClassifier_Final/artifacts/models/bert_toxic


In [ ]:
# from transformers import BertForSequenceClassification
# 
# def load_model(output_dir):
#     try:
#         print(f"→ Loading model from {output_dir}…")
#         return BertForSequenceClassification.from_pretrained(
#             output_dir,
#             local_files_only=True,
#             num_labels=6
#         )
#     except Exception as e:
#         print(f"⚠️  Failed to load from {output_dir}: {e}")
#         print("→ Falling back to `bert-base-uncased` from the Hub…")
#         return BertForSequenceClassification.from_pretrained(
#             "bert-base-uncased",
#             num_labels=6
#         )
# 
# model = load_model(output_dir)


NameError: name 'output_dir' is not defined

In [ ]:
!ls ./ToxicCommentClassifier_Final/artifacts/models/bert_toxic/

checkpoint-150	config.json	   runs			    tokenizer.json
checkpoint-200	model.safetensors  special_tokens_map.json  vocab.txt
checkpoint-250	pytorch_model.bin  tokenizer_config.json


In [ ]:
# model_path = "./ToxicCommentClassifier_Final/artifacts/models/bert_toxic/"


In [ ]:
# from transformers import AutoModelForSequenceClassification
# 
# model = AutoModelForSequenceClassification.from_pretrained(
#     "./ToxicCommentClassifier_Final/artifacts/models/bert_toxic/",
#     local_files_only=True
# )


In [2]:
import os
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    BertForSequenceClassification,
    BertTokenizerFast,
    Trainer,
    TrainingArguments,
    trainer_utils
)

def load_model_or_fallback(path, num_labels=6):
    if os.path.isdir(path) and os.path.isfile(os.path.join(path, "pytorch_model.bin")):
        print(f"→ Loading model from {path}")
        return BertForSequenceClassification.from_pretrained(
            path, local_files_only=True, num_labels=num_labels
        )
    else:
        print("⚠️  Checkpoint not found — falling back to bert-base-uncased")
        return BertForSequenceClassification.from_pretrained(
            "bert-base-uncased", num_labels=num_labels
        )


def find_valid_checkpoint(base_dir):
    candidates = sorted(glob.glob(f"{base_dir}/checkpoint-*"), key=os.path.getmtime)
    for ckpt in reversed(candidates):
        # check for any of the known weight files
        if (
            os.path.isfile(os.path.join(ckpt, "pytorch_model.bin")) or
            os.path.isfile(os.path.join(ckpt, "model.safetensors"))
        ):
            return ckpt
    return None

def main():
    model_dir  = "./ToxicCommentClassifier_Final/artifacts/models/bert_toxic"
    data_path  = "./ToxicCommentClassifier_Final/artifacts/train.csv"
    os.makedirs(model_dir, exist_ok=True)

    # 1) Load tokenizer + model (or fallback)
    tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", use_fast=True)
    model     = load_model_or_fallback(model_dir, num_labels=6)

    # 2) Read CSV via pandas + wrap in a Dataset
    df = pd.read_csv(data_path)
    ds = Dataset.from_pandas(df)
    
    # 3) Tokenize + attach labels
    def preprocess(ex):
        toks = tokenizer(
            ex["comment_text"],
            truncation=True,
            padding="max_length",
            max_length=128
        )
        toks["labels"] = ex["toxic"]
        return toks

    tokenized = ds.map(
        preprocess,
        batched=True,
        remove_columns=ds.column_names
    )

    # 4) TrainingArguments
    training_args = TrainingArguments(
        output_dir=     model_dir,
        num_train_epochs=3,
        per_device_train_batch_size=16,
        learning_rate=    2e-5,
        warmup_steps=      100,
        logging_steps=     100,
        save_strategy=   "steps",   # Save every X steps
        save_steps=        100,
        save_total_limit=   2,
        fp16= torch.cuda.is_available(),
    )

    # 5) Trainer
    trainer = Trainer(
        model= model,
        args=  training_args,
        train_dataset=tokenized,
        tokenizer=    tokenizer,
    )

    # 6) Resume from checkpoint only if valid folder exists
    last_ckpt = trainer_utils.get_last_checkpoint(model_dir)

    # Check if it's a valid directory with actual model files
    if last_ckpt and os.path.isdir(last_ckpt) and os.path.isfile(os.path.join(last_ckpt, "pytorch_model.bin")):
        print(f"→ Resuming from checkpoint {last_ckpt}")
        resume = last_ckpt
    else:
        print("⚠️  No valid checkpoint folder found, starting fresh")
        resume = None


    # 7) Train
    trainer.train(resume_from_checkpoint=resume)

    # 8) Save final
    trainer.save_model(model_dir)
    tokenizer.save_pretrained(model_dir)
    print("✓ Done! Model saved to", model_dir)

if __name__ == "__main__":
    main()


→ Loading model from ./ToxicCommentClassifier_Final/artifacts/models/bert_toxic


Map:   0%|          | 0/159571 [00:00<?, ? examples/s]

Exception ignored in: <function tqdm.__del__ at 0x000001C07D48C430>
Traceback (most recent call last):
  File "D:\Users\ved\anaconda3\envs\AI_F2_torch\lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
KeyboardInterrupt: 


⚠️  No valid checkpoint folder found, starting fresh


Step,Training Loss
100,0.741800
200,0.160100
300,0.117300
400,0.115100
500,0.138400
600,0.113200
700,0.146400
800,0.124100
900,0.120500
1000,0.107900


✓ Done! Model saved to ./ToxicCommentClassifier_Final/artifacts/models/bert_toxic


In [1]:
# test.py

# 0) Install & mount (run once in a fresh Colab cell)
# !pip install transformers datasets scikit-learn tqdm --quiet
# from google.colab import drive
# drive.mount('/content/drive')

import os, re
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
from transformers import BertTokenizerFast, BertForSequenceClassification
from sklearn.metrics import f1_score, recall_score, hamming_loss

# ─── CONSTANTS ──────────────────────────────────────────
LABELS    = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]
MODEL_DIR = "./ToxicCommentClassifier_Final/artifacts/models/bert_toxic"
TEST_CSV  = "./ToxicCommentClassifier_Final/artifacts/test.csv"
OUT_CSV   = "./ToxicCommentClassifier_Final/artifacts/test_predicted.csv"
BATCH     = 256

# ─── SETUP ───────────────────────────────────────────────
device    = "cuda" if torch.cuda.is_available() else "cpu"
print(f"→ Using device: {device}")

print("→ Loading tokenizer + model…")
tokenizer = BertTokenizerFast.from_pretrained(MODEL_DIR, local_files_only=True)
model     = BertForSequenceClassification.from_pretrained(
    MODEL_DIR,
    local_files_only=True,
    num_labels=len(LABELS)
).to(device).eval()

# ─── UTILITIES ───────────────────────────────────────────
def predict_batch(texts):
    enc = tokenizer(
        texts,
        max_length=128,
        padding="longest",
        truncation=True,
        return_tensors="pt"
    ).to(device)
    with torch.no_grad():
        logits = model(**enc).logits.cpu().numpy()
    # sigmoid → probabilities
    return 1 / (1 + np.exp(-logits))

def blur_if_needed(text, probs, thr=0.5):
    if probs[0]>=thr or probs[2]>=thr or probs[4]>=thr:
        bad = ["fuck","shit","bitch","idiot","asshole","bastard","retard","dumb"]
        pattern = r"\b(" + "|".join(map(re.escape,bad)) + r")\b"
        return re.sub(pattern, "****", text, flags=re.IGNORECASE)
    return text

def format_probs(probs):
    # returns a single string block of six lines
    lines = [f"{lbl:<15s}: {p:.2f}" for lbl,p in zip(LABELS, probs)]
    return "\n".join(lines)

# ─── LOAD & PREDICT ─────────────────────────────────────
print(f"→ Loading test set from {TEST_CSV}")
df = pd.read_csv(
    TEST_CSV,
    encoding="latin1",        # ISO-8859-1 will accept all bytes
    engine="python",          # fall back to the Python parser (more flexible)
    on_bad_lines="skip"       # (optional) skip any malformed lines
)
print(f"   {len(df)} rows found.\n")


all_probs = []
all_preds = []

print("→ Running predictions:")
for i in tqdm(range(0, len(df), BATCH), desc="batches"):
    batch = df["comment_text"].iloc[i : i+BATCH].tolist()
    probs = predict_batch(batch)
    preds = (probs >= 0.5).astype(int)
    all_probs.append(probs)
    all_preds.append(preds)

all_probs = np.vstack(all_probs)
all_preds = np.vstack(all_preds)

# ─── OVERALL METRICS ────────────────────────────────────
y_true = df[LABELS].values
y_pred = all_preds

macro_f1     = f1_score(y_true, y_pred, average="macro", zero_division=0)
macro_recall = recall_score(y_true, y_pred, average="macro", zero_division=0)
h_loss       = hamming_loss(y_true, y_pred)

print("\n=== Overall evaluation metrics ===")
print(f" Macro-F1     : {macro_f1:.4f}")
print(f" Macro-Recall : {macro_recall:.4f}")
print(f" Hamming-Loss : {h_loss:.4f}\n")

# ─── BUILD OUTPUT DF ────────────────────────────────────
print("→ Building output DataFrame and saving CSV…")
out = {
    "id":              df["id"],
    "comment_text":    df["comment_text"],
    "predicted_label": [],
    "blurred_text":    [],
    "probabilities":   [],
    "metrics":         []
}

for idx, txt in enumerate(df["comment_text"]):
    p      = all_probs[idx]
    pred   = all_preds[idx]
    # comma list of active labels
    chosen = [lbl for lbl,fl in zip(LABELS, pred) if fl]
    label = ",".join(chosen) if chosen else "none"
    out["predicted_label"].append(label)
    out["blurred_text"].append(blur_if_needed(txt, p))
    out["probabilities"].append(format_probs(p))
    # same metrics block on every row
    out["metrics"].append(
        f"F1={macro_f1:.3f}  Recall={macro_recall:.3f}  HLoss={h_loss:.3f}"
    )

out_df = pd.DataFrame(out)
out_df.to_csv(OUT_CSV, index=False)

print(f"→ Saved predictions + probs + metrics → {OUT_CSV}")
print("✅ All done.")


→ Using device: cuda
→ Loading tokenizer + model…
→ Loading test set from ./ToxicCommentClassifier_Final/artifacts/test.csv
   153164 rows found.

→ Running predictions:


batches:   0%|          | 0/599 [00:00<?, ?it/s]

D:\Users\ved\anaconda3\envs\AI_F2_torch\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


KeyError: "None of [Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',\n       'identity_hate'],\n      dtype='object')] are in the [columns]"

In [5]:
MODEL_DIR = "./ToxicCommentClassifier_Final/artifacts/models/bert_toxic"
TEST_CSV  = "./ToxicCommentClassifier_Final/artifacts/test.csv"
OUT_CSV   = "./ToxicCommentClassifier_Final/artifacts/test_predicted.csv"


In [8]:
OUT_CSV   = TEST_CSV.replace(".csv", "_predicted2rev.csv")


In [9]:
import os, io, re, sys
import torch
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from transformers import BertTokenizerFast, BertForSequenceClassification
from sklearn.metrics import f1_score, recall_score, hamming_loss

LABELS = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
THRESH = 0.5       # threshold for binary preds & masking
BATCH  = 256       # for sentence‐level batches
WORD_BATCH = 64    # for word‐level masking batches

# ─── UTILITIES ─────────────────────────────────────────────

def load_test_df(path):
    """Read CSV with multiple encoding fallbacks."""
    print(f"→ Loading test set from {path}", file=sys.stderr)
    for enc in ("utf-8", "cp1252"):
        try:
            df = pd.read_csv(path, encoding=enc, on_bad_lines="skip")
            print(f"   OK with encoding={enc!r}", file=sys.stderr)
            return df
        except UnicodeDecodeError:
            print(f"   failed {enc!r}, trying next…", file=sys.stderr)
    # final fallback
    print("   fallback: errors='replace'", file=sys.stderr)
    raw = open(path, "r", encoding="utf-8", errors="replace").read()
    df = pd.read_csv(io.StringIO(raw), on_bad_lines="skip")
    print("   fallback loaded", file=sys.stderr)
    return df

def blur_offensive(text, tokenizer, model, device, thresh=0.5, batch_size=64):
    """
    Split `text` into whitespace‐words, run each word through BERT,
    and mask only those whose *own* toxic‐score > thresh.
    """
    words = text.split()
    if not words:
        return text

    to_mask = set()
    # slide window over words
    for i in range(0, len(words), batch_size):
        batch_words = words[i : i + batch_size]
        enc = tokenizer(
            batch_words,
            truncation=True,
            padding="longest",
            max_length=64,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            logits = model(**enc).logits      # shape [batch, 6]
            probs  = torch.sigmoid(logits)    # same shape

        toxic_scores = probs[:, 0].cpu().numpy()
        for j, score in enumerate(toxic_scores):
            if score > thresh:
                to_mask.add(i + j)

    # rebuild string, masking only flagged words
    out = []
    for idx, w in enumerate(words):
        out.append("****" if idx not in to_mask else w)
    return " ".join(out)

# ─── MAIN ────────────────────────────────────────────────────

def main():
    # 1) load tokenizer + model
    print("→ Loading tokenizer + model…", file=sys.stderr)
    tok = BertTokenizerFast.from_pretrained(MODEL_DIR, local_files_only=True)
    mdl = BertForSequenceClassification.from_pretrained(
        MODEL_DIR, local_files_only=True, num_labels=len(LABELS)
    ).to(DEVICE).eval()

    # 2) load test set
    df = load_test_df(TEST_CSV)
    print(f"   {len(df)} rows; columns = {list(df.columns)}\n", file=sys.stderr)

    ids   = df["id"].astype(str).tolist()
    texts = df["comment_text"].fillna("").tolist()

    # 3) sentence‐level batch predictions
    print("→ Running sentence-level predictions…", file=sys.stderr)
    all_probs = []
    for i in tqdm(range(0, len(texts), BATCH), file=sys.stderr):
        batch = texts[i : i + BATCH]
        enc   = tok(
            batch,
            padding="longest",
            truncation=True,
            max_length=128,
            return_tensors="pt"
        ).to(DEVICE)
        with torch.no_grad():
            logits = mdl(**enc).logits
            probs  = torch.sigmoid(logits).cpu().numpy()
        all_probs.append(probs)
    all_probs = np.vstack(all_probs)

    # 4) threshold → binary
    all_preds = (all_probs >= THRESH).astype(int)

    # 5) compute evaluation if ground-truth present
    gt = [c for c in LABELS if c in df.columns]
    if len(gt) == len(LABELS):
        y_true = df[gt].fillna(0).astype(int).values
        f1m    = f1_score(y_true, all_preds, average="macro", zero_division=0)
        recm   = recall_score(y_true, all_preds, average="macro", zero_division=0)
        hloss  = hamming_loss(y_true, all_preds)
        print(
            f"\n=== EVALUATION ===\n"
            f"Macro-F1     : {f1m:.4f}\n"
            f"Macro-Recall : {recm:.4f}\n"
            f"Hamming-Loss : {hloss:.4f}\n",
            file=sys.stderr
        )
    else:
        print("→ No ground-truth columns found, skipping metrics.", file=sys.stderr)
        f1m = recm = hloss = float("nan")

    # 6) build output DataFrame
    out = pd.DataFrame({
        "id":             ids,
        "comment_text":   texts,
        "predicted_labels":[
            ",".join(LABELS[j] for j, bit in enumerate(row) if bit)
            for row in all_preds
        ],
        "blurred_text": [
            blur_offensive(txt, tok, mdl, DEVICE, thresh=THRESH, batch_size=WORD_BATCH)
            for txt in texts
        ]
    })

    # attach per-label probabilities
    for i, lbl in enumerate(LABELS):
        out[f"prob_{lbl}"] = all_probs[:, i].round(3)

    # attach global metrics
    out["f1_macro"]     = f1m
    out["recall_macro"] = recm
    out["hamming_loss"] = hloss

    # 7) save
    print(f"→ Writing predictions to {OUT_CSV}", file=sys.stderr)
    out.to_csv(OUT_CSV, index=False)
    print("✓ Done.", file=sys.stderr)


if __name__ == "__main__":
    main()



→ Loading tokenizer + model…
→ Loading test set from ./ToxicCommentClassifier_Final/artifacts/test.csv
   OK with encoding='utf-8'
   999 rows; columns = ['id', 'comment_text']

→ Running sentence-level predictions…


  0%|          | 0/4 [00:00<?, ?it/s]

→ No ground-truth columns found, skipping metrics.
→ Writing predictions to ./ToxicCommentClassifier_Final/artifacts/test_predicted2rev.csv
✓ Done.
